# Purpose

Weighter is designed to be a hacky weight tracking app using Slack as a frontend and Google Sheets as a database! 
Weights are entered through a Slack Channel, stored in a Google Sheet, and reported back to users through Slack. Users will have the option to view various stats and graphs by sending different slack messages. 

Weighter also features additive modeling forecasts using the Facebook Prophet library. 

## Setup Libraries and Access to the Google Sheet

In [64]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# fbprophet for additive models
import fbprophet

# gspread for Google Sheets access
import gspread

# slacker for interacting with Slack
from slacker import Slacker

# oauth2client for authorizing access to Google Sheets
from oauth2client.service_account import ServiceAccountCredentials

# os for deleting images
import os

In [65]:
# matplotlib for plotting in the notebook
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib

### Google Sheet Access

The json file is the credentials for accessing the google sheet generated from the Google Developers API. To access a specific sheet, you need to share the sheet with the email address in the json file. 

In [66]:
# google sheets access
scope = ['https://spreadsheets.google.com/feeds']

# Use local stored credentials in json file
# make sure to first share the sheet with the email in the json file
credentials = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/Will Koehrsen/Desktop/weighter-2038ffb4e5a6.json', scope)

# Authorize access
gc = gspread.authorize(credentials);

INFO:oauth2client.client:Refreshing access_token


## Set up Slack Access

In [67]:
# Slack api key is stored as text file
with open('C:/Users/Will Koehrsen/Desktop/slack_api.txt', 'r') as f:
    slack_api_key = f.read()

In [68]:
slack = Slacker(slack_api_key)

In [69]:
slack.chat.post_message('#test_python', 'Hello Fellow Slackers')

### Open the sheet and convert to a pandas dataframe

In [75]:
# Open the sheet, need to share the sheet with email specified in json file
gsheet = gc.open('Auto Weight Challenge').sheet1

# List of lists with each row in the sheet as a list
weight_lists = gsheet.get_all_values()

# Headers are the first list
# Pop returns the element (list in this case) and removes it from the list
headers = weight_lists.pop(0)

# Convert list of lists to a dataframe with specified column header
weights = pd.DataFrame(weight_lists, columns=headers)

# Record column should be a boolean
weights['Record'] = weights['Record'].astype(bool)

# Name column is a string
weights['Name'] = weights['Name'].astype(str)

# Convert dates to datetime, then set as index, then set the time zone
weights['Date'] = pd.to_datetime(weights['Date'], unit='s')
weights  = weights.set_index('Date', drop = True).tz_localize(tz='US/Eastern')

# Drop any extra entries
weights = weights.drop('NaT')

In [76]:
weights.head()

,Name,Entry,Record
Date,,,
2017-08-18 00:00:00-04:00,koehrcl,235.2,True
2017-08-19 00:00:00-04:00,koehrcl,235.6,True
2017-08-20 00:00:00-04:00,koehrcl,233,True
2017-08-21 00:00:00-04:00,koehrcl,232.6,True
2017-08-22 00:00:00-04:00,koehrcl,234.4,True


In [77]:
weights.tail()

,Name,Entry,Record
Date,,,
2018-01-19 15:30:25-05:00,willkoehrsen,137.3,True
2018-01-19 18:11:49-05:00,fletcher,188.4,True
2018-01-20 15:39:12-05:00,willkoehrsen,137,True
2018-01-20 15:49:52-05:00,koehrcl,220.4,True
2018-01-20 18:09:57-05:00,fletcher,188.6,True


+ Date is the index (in Eastern time here)
+ Name is the slack username
+ Entry is either weight or a string to display results
+ Record is whether or not the entry has been processed by weighter

# Weighter Class

The class will include a number of different methods for analyzing the data and graphing results. These results can then be sent back to Slack depending on the message entered by the user.

In [78]:
class Weighter():
    
    """
    When weighter is initialized, we need to convert the usernames,
    get a dictionary of the unrecorded entries, construct a dictionary
    of the actions to take, and make sure all data is formatted correctly
    """
    
    def __init__(self, weights):
        
        # Weights is a dataframe
        self.weights = weights.copy()
        
        # Users is a list of the unique users in the data
        self.users = list(set(self.weights['Name']))
        
        correct_names = []
        
        # Name Changes
        for user in self.weights['Name']:
            
            # Have to hardcode in name Changes
            if user == 'koehrcl':
                correct_names.append('Craig')
            elif user == 'willkoehrsen':
                correct_names.append('Will')
            elif user == 'fletcher':
                correct_names.append('Fletcher')
            
            # Currently do not handle new users
            else:
                print('New User Detected')
                return
            
        self.weights['Name'] = correct_names
        
        # Users is a list of the unique users in the data
        self.users = list(set(self.weights['Name']))
        
        # Create a dataframe of the unrecorded entries
        self.unrecorded = self.weights[self.weights['Record'] != True]
        
        # Process the unrecorded entries
        self.process_unrecorded()
        
        # The remaning entries will all be weights
        self.weights['Entry'] = [float(weight) for weight in self.weights['Entry']]
        
        # Build the user dictionary
        self.build_user_dict()
        
        self.calculate_columns()
        
    """ 
    Constructs a dictionary for each user with critical information
    This forms the basis for the summarize function
    """
    
    def build_user_dict(self):
        
        user_dict = {}
        
        user_goals = {'Craig': 215.0, 'Fletcher': 200.0, 'Will': 155.0}
        user_colors = {'Craig': 'forestgreen', 'Fletcher': 'navy', 'Will': 'darkred'}
        
        for i, user in enumerate(self.users):
            
            user_weights = self.weights[self.weights['Name'] == user]
            goal = user_goals.get(user)

            start_weight = user_weights.ix[min(user_weights.index), 'Entry']   
            start_date = min(user_weights.index)
            
            # Find minimum weight and date on which it occurs
            min_weight =  min(user_weights['Entry'])
            min_weight_date = ((user_weights[user_weights['Entry'] == min_weight].index)[0])
            
            # Find maximum weight and date on which it occurs
            max_weight = max(user_weights['Entry'])
            max_weight_date = ((user_weights[user_weights['Entry'] == max_weight].index)[0])
            
            most_recent_weight = user_weights.ix[max(user_weights.index), 'Entry']
            
            if goal < start_weight:
                change = start_weight - most_recent_weight
                obj = 'lose'
            elif goal > start_weight:
                change = most_recent_weight - start_weight
                obj = 'gain'
                
            pct_change = 100 * change / start_weight
            
            pct_to_goal = 100 * (change / abs(start_weight - goal) )
            
            # Color for plotting
            user_color = user_colors[user]
            
            user_dict[user] = {'min_weight': min_weight, 'max_weight': max_weight,
                               'min_date': min_weight_date, 'max_date': max_weight_date,
                               'recent': most_recent_weight, 'abs_change': change,
                               'Pct_change': pct_change, 'pct_towards_goal': pct_to_goal,
                               'start_weight': start_weight, 'start_date': start_date,
                               'goal_weight': goal, 'objective': obj, 'color': user_color}
       
        self.user_dict = user_dict
             
    """
    Builds a dictionary of unrecorded entries where each key is the user
    and the value is a list of weights and methods called for by the user.
    This dictionary is saved as the entries attribute of the class.
    Removes the none weights from the data and from the google sheet.
    """
    
    def process_unrecorded(self):
        
        entries = {name:[] for name in self.users}
        drop = []
        
        location = {}
        for index in self.unrecorded.index:

            entry = self.unrecorded.ix[index, 'Entry']
            user = str(self.unrecorded.ix[index, 'Name'])
            
            # Try and except does not seem like the best way to handle this
            try:
                entry = float(entry)
                entries[user].append(entry)
                location[index] = True
                
            except:  
                entry = str(entry)
                entries[user].append(entry)
                location[index] = 'remove'
                
                drop.append(index)
                
            self.weights.ix[index, 'Record'] = True
           
        self.location = location
        self.update_sheet()
        # Drop the rows which do not contain a weight
        self.weights.drop(drop, axis=0, inplace=True)

        # Entries is all of the new entries
        self.entries = entries
        
        
    def update_sheet(self):
        delete_count = 0
        
        # Iterate through the locations and update as appropriate
        for index, action in self.location.items():
            cell_row = (np.where(self.weights.index == index))[0][0] + 2 - delete_count
            if action == 'remove':
                gsheet.delete_row(index = cell_row)
                delete_count += 1
            elif action:
                gsheet.update_acell(label='D%d' % cell_row, val = 'True')
           
    """ 
    Iterates through the unrecorded entries and delegates 
    each one to the appropriate method.
    Updates the record cell in the google sheet 
    """
    def process_entries(self):
        for user, user_entries in self.entries.items():
            for entry in user_entries:
                if type(entry) == float:
                    self.basic_message(user)
                
                elif entry.lower() == 'summary':
                    self.summary(user)
                    
                elif entry.lower() == 'percent':
                    self.percentage_plot()
                    
                elif entry.lower() == 'history':
                    self.history_plot(user)
                
    """ 
    This method is automatically run for each new weight
    """
    def basic_message(self, user):
    
        # Find information for user, construct message, post message to Slack
        user_info = self.user_dict.get(user)

        message = ("\n{}: Total Weight Change = {:.2f} lbs.\n\n"
                    "Percentage Weight Change = {:.2f}%").format(user, user_info['abs_change'],
                                                     user_info['Pct_change'])

        slack.chat.post_message('#test_python', text=message, username='Weight Challenge Update')
                        
    """ 
    Displays comprehensive stats about the user
    Only run on a summary message in the slack channel
    """
    
    def summary(self, user):
        user_info = self.user_dict.get(user)
        message = ("\n{}, your most recent weight was {:.2f} lbs.\n\n"
                   "Absolute weight Change = {:.2f} lbs, percentage weight Change = {:.2f}%.\n\n"
                   "Minimum weight = {:.2f} lbs on {} and maximum weight = {:.2f} lbs on {}.\n\n"
                   "Your goal weight = {:.2f} lbs. and you are {:.2f}% of the way there.\n\n"
                   "You started at {:.2f} lbs on {}. Congratulations on the progress!").format(user, 
                     user_info['recent'], user_info['abs_change'], user_info['Pct_change'], 
                     user_info['min_weight'], str(user_info['min_date'].date()),
                     user_info['max_weight'], str(user_info['max_date'].date()),
                     user_info['goal_weight'], user_info['pct_towards_goal'],                                                       
                     user_info['start_weight'], str(user_info['start_date'].date()))
        
        slack.chat.post_message('#test_python', text=message, username='%s Summary' % user)
        
    """ 
    Adds the change and percentage change columns to the self.weights df
    """
    def calculate_columns(self):
        
        self.weights = self.weights.sort_values('Name')
        self.weights['Change'] = 0
        self.weights['Pct_change'] = 0
        self.weights.reset_index(level=0, inplace = True)
        
        for index in self.weights.index:
            user = self.weights.ix[index, 'Name']
            weight = self.weights.ix[index, 'Entry']
            start_weight = self.user_dict[user]['start_weight']
            objective = self.user_dict[user]['objective']
            
            if objective == 'lose':
                
                self.weights.ix[index, 'Change'] = start_weight - weight
                self.weights.ix[index, 'Pct_change'] = 100 * (start_weight - weight) / start_weight
                
            elif objective == 'gain':
                self.weights.ix[index, 'Change'] = weight - start_weight
                self.weights.ix[index, 'Pct_change'] = 100 * (weight - start_weight) / start_weight

        self.weights.set_index('Date', drop=True, inplace=True)
        
    @staticmethod
    def reset_plot():
        matplotlib.rcParams.update(matplotlib.rcParamsDefault)
        
    def percentage_plot(self):
        
        self.reset_plot()
        
        plt.style.use('fivethirtyeight')
        plt.figure(figsize=(10,8))

        for i, user in enumerate(weighter.users):
            
            user_color = self.user_dict[user]['color']

            # Select the user and order dataframe by date
            df = self.weights[self.weights['Name'] == user]
            df.sort_index(inplace=True)
            
            # List is used for fitting polynomial
            xvalues = list(range(len(df)))

            # Create a polynomial fit
            z = np.polyfit(xvalues, df['Pct_change'], deg=6)

            # Create a function from the fit
            p = np.poly1d(z)

            # Map the x values to y values
            fit_data = p(xvalues)

            # Plot the actual points and the fit
            plt.plot(df.index, df['Pct_change'], 'o', color = user_color, label = '%s' % user)
            plt.plot(df.index, fit_data, '-', color = user_color, linewidth = 5, label = '%s' % user)


        # Plot formatting
        plt.xlabel('Date'); plt.ylabel('Percentage Change')
        plt.title('Percentage Changes')
        plt.grid(color='k', alpha=0.4)
        plt.legend(prop={'size':14})
        plt.savefig(fname='percentage_plot.png');
        
        slack.files.upload('percentage_plot.png', channels='#test_python')
        
        os.remove('percentage_plot.png')
        
    def history_plot(self, user):
        
        self.reset_plot()
        plt.style.use('fivethirtyeight')
        plt.figure(figsize=(10, 8))
        
        df = self.weights[self.weights['Name'] == user]
        df.sort_index(inplace=True) 
        user_color = self.user_dict[user]['color']
        
        # List is used for fitting polynomial
        xvalues = list(range(len(df)))

        # Create a polynomial fit
        z = np.polyfit(xvalues, df['Entry'], deg=6)

        # Create a function from the fit
        p = np.poly1d(z)

        # Map the x values to y values
        fit_data = p(xvalues)

        # Make a simple plot and upload to slack
        plt.plot(df.index, df['Entry'], 'ko', ms = 8, label = 'Observed')
        plt.plot(df.index, fit_data, '-', color = user_color, linewidth = 5, label = 'Smooth Fit')
        plt.xlabel('Date'); plt.ylabel('Weight (lbs)'); plt.title('%s Weight History' % user)
        plt.legend(prop={'size': 14});
        
        plt.savefig(fname='history_plot.png')
        slack.files.upload('history_plot.png', channels='#test_python')
        
        os.remove('history_plot.png')

In [79]:
weighter = Weighter(weights)

In [80]:
update = weighter.weights

In [81]:
update.tail()

,Name,Entry,Record,Change,Pct_change
Date,,,,,
2017-11-27 14:20:27-05:00,Will,134.8,True,8.2,6.477093
2017-09-29 00:00:00-04:00,Will,134.6,True,8.0,6.319115
2017-09-28 00:00:00-04:00,Will,134.5,True,7.9,6.240126
2017-11-30 03:02:49-05:00,Will,134.8,True,8.2,6.477093
2017-10-23 00:00:00-04:00,Will,134.7,True,8.1,6.398104


In [82]:
update.head()

,Name,Entry,Record,Change,Pct_change
Date,,,,,
2017-08-18 00:00:00-04:00,Craig,235.2,True,0.0,0.000000
2018-01-01 15:03:18-05:00,Craig,220.8,True,14.4,6.122449
2018-01-02 14:39:48-05:00,Craig,221.6,True,13.6,5.782313
2018-01-06 15:10:54-05:00,Craig,219.8,True,15.4,6.547619
2018-01-09 13:40:09-05:00,Craig,221.2,True,14.0,5.952381


In [55]:
(np.where(update.index == key))[0][0]

274

In [39]:
update.tail(12)

,Name,Entry,Record,Change,Pct_change
Date,,,,,
2018-01-17 15:54:44-05:00,Craig,222.2,True,13.0,5.53
2018-01-17 15:55:47-05:00,Fletcher,188.0,True,3.4,1.84
2018-01-18 19:46:11-05:00,Will,137.8,True,11.2,8.85
2018-01-18 20:21:28-05:00,Craig,220.2,True,15.0,6.38
2018-01-18 21:55:45-05:00,Fletcher,188.6,True,4.0,2.17
2018-01-19 14:34:56-05:00,Craig,221.1,True,14.1,5.99
2018-01-19 15:30:25-05:00,Will,137.3,True,10.7,8.45
2018-01-19 18:11:49-05:00,Fletcher,188.4,True,3.8,2.06
2018-01-20 15:39:12-05:00,Will,137.0,True,10.4,8.21


In [40]:
weighter.unrecorded

,Name,Entry,Record,Change,Pct_change
Date,,,,,
2018-01-18 19:46:11-05:00,Will,137.8,False,11.2,8.85
2018-01-18 20:21:28-05:00,Craig,220.2,False,15,6.38
2018-01-18 20:44:55-05:00,Will,Summary,False,,
2018-01-18 20:45:01-05:00,Will,Future,False,,
2018-01-18 21:55:45-05:00,Fletcher,188.6,False,4,2.17
2018-01-19 14:34:56-05:00,Craig,221.1,False,14.1,5.99
2018-01-19 15:30:25-05:00,Will,137.3,False,10.7,8.45
2018-01-19 18:11:49-05:00,Fletcher,188.4,False,3.8,2.06
2018-01-20 15:39:12-05:00,Will,137,False,10.4,8.21


In [ ]:
weighter.percentage_plot()

In [ ]:
gsheet.update_cell(row= 1, col = 6, val = 0)

In [ ]:
weighter.history_plot('Will')

In [ ]:
weighter.percentage_plot()

In [ ]:
weights_craig = weighter.weights[weighter.weights['Name'] == 'Craig']
weights_craig.sort_index(inplace=True)
plt.plot(weights_craig.index, weights_craig['Change']);

In [ ]:
we = weighter.weights

In [ ]:
matplotlib.rcParams.update(matplotlib.rcParamsDefault)

In [ ]:
plt.figure(figsize=(10,8))
colors = ['forestgreen', 'navy', 'darkred']
plt.style.use('fivethirtyeight')
for i, user in enumerate(weighter.users):
    
    
    df = we[we['Name'] == user]
    df.sort_index(inplace=True)
    xvalues = list(range(len(df)))
    
    # Create a polynomial fit
    z = np.polyfit(xvalues, df['Pct_change'], deg=5)
    
    # Create a function from the fit
    p = np.poly1d(z)
    
    # Map the x values to y values
    fit_data = p(xvalues)
    
    # Plot the actual points and the fit
    plt.plot(df.index, df['Pct_change'], 'o', color = colors[i], label = '%s' % user)
    plt.plot(df.index, fit_data, '-', color = colors[i], linewidth = 4, label = '%s' % user)
    
    

plt.xlabel('Date'); plt.ylabel('Percentage Change')
plt.title('Percentage Changes')
plt.grid(color='k', alpha=0.4)
plt.legend(prop={'size':14})
plt.show()

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
info, z = curve_fit(func, list(range(len(df.index))), df['Pct_change'], maxfev = 10000)

In [ ]:
z = np.polyfit(list(range(len(df.index))), df['Pct_change'], deg = 4)

In [ ]:
p = np.poly1d(z)
p(df['Pct_change'])

In [ ]:
Change,Pct_change = weighter.calculate_columns()

In [ ]:
Change = np.array(Change)

In [ ]:
Change = Change.reshape((-1, ))

In [ ]:
weighter.weights['Change'] = Change

In [ ]:
weighter.weights

In [ ]:
flat = [j for entry in Change for j in entry]

In [ ]:
flat

In [ ]:
weighter.weights['Change'] = Change

In [ ]:

weighter.weights.tail()

In [ ]:
weighter.process_entries()

In [ ]:
weighter.summary('Craig')

In [ ]:
weighter.summary('Craig')